In [1]:
# Import modules

import warnings

# import holoviews as hv
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import numpy as np
import shapely
import pandas as pd
import pystac_client
import xarray as xr
import rioxarray as rio
import pathlib
from pathlib import Path
import pystac
import pystac_client
import geopandas as gpd

#import colormaps as cmaps
import pyam # https://pyam-iamc.readthedocs.io/en/latest/index.html
import matplotlib.colors as mcolors

from copy import deepcopy
from typing import List, Dict

# Import custom functionality
from coclicodata.drive_config import p_drive

# ignore warnings
warnings.filterwarnings("ignore")

C:\Users\soest\AppData\Local\Temp\ipykernel_9948\2054427804.py:20: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


<IPython.core.display.Javascript object>

In [61]:
# Convert NUTS to parquet file

fp = Path(r"p:\11207608-coclico\FASTTRACK_DATA\XX_NUTS\NUTS_RG_01M_2021_3035.shp")

NUTS_data = gpd.read_file(fp)

NL_NUTS = NUTS_data.loc[NUTS_data.CNTR_CODE == 'NL']

NL_NUTS

,NUTS_ID,LEVL_CODE,CNTR_CODE,NAME_LATN,NUTS_NAME,MOUNT_TYPE,URBN_TYPE,COAST_TYPE,geometry
34,NL,0,NL,Nederland,Nederland,0.0,0,0,"MULTIPOLYGON (((4105407.002 3377924.206, 41082..."
84,NL4,1,NL,Zuid-Nederland,Zuid-Nederland,0.0,0,0,"MULTIPOLYGON (((4013364.197 3199017.072, 40135..."
121,NL3,1,NL,West-Nederland,West-Nederland,0.0,0,0,"MULTIPOLYGON (((3996956.058 3331611.877, 39992..."
138,NL2,1,NL,Oost-Nederland,Oost-Nederland,0.0,0,0,"POLYGON ((4059751.046 3312033.418, 4061769.361..."
146,NL1,1,NL,Noord-Nederland,Noord-Nederland,0.0,0,0,"MULTIPOLYGON (((4105407.002 3377924.206, 41082..."
283,NL12,2,NL,Friesland (NL),Friesland (NL),0.0,0,0,"MULTIPOLYGON (((4065534.649 3379840.831, 40653..."
284,NL13,2,NL,Drenthe,Drenthe,0.0,0,0,"POLYGON ((4086199.481 3349628.157, 4086849.505..."
289,NL11,2,NL,Groningen,Groningen,0.0,0,0,"MULTIPOLYGON (((4105407.002 3377924.206, 41082..."
291,NL41,2,NL,Noord-Brabant,Noord-Brabant,0.0,0,0,"MULTIPOLYGON (((4013364.197 3199017.072, 40135..."
292,NL42,2,NL,Limburg (NL),Limburg (NL),0.0,0,0,"POLYGON ((4040141.102 3189287.625, 4040613.782..."


In [18]:
# Load LAU data

fp = Path(r"p:\11207608-coclico\FASTTRACK_DATA\XX_LAU\LAU_RG_01M_2020_3035.parquet")

LAU_data = gpd.read_parquet(fp)

NL_LAU = LAU_data.loc[LAU_data.CNTR_CODE == 'NL']
NL_LAU

,GISCO_ID,CNTR_CODE,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
82836,NL_GM0715,NL,GM0715,Terneuzen,54438.0,208.581512,260.991491,2020,NL_GM0715,"POLYGON ((3902942.028 3142804.928, 3901429.640..."
82837,NL_GM0716,NL,GM0716,Tholen,25758.0,157.189707,163.865691,2020,NL_GM0716,"POLYGON ((3919494.322 3185441.523, 3920338.623..."
82838,NL_GM0717,NL,GM0717,Veere,21885.0,154.933571,141.254086,2020,NL_GM0717,"MULTIPOLYGON (((3886262.003 3189064.122, 38865..."
82839,NL_GM0718,NL,GM0718,Vlissingen,44365.0,1259.406598,35.226908,2020,NL_GM0718,"MULTIPOLYGON (((3884403.974 3168105.183, 38840..."
82841,NL_GM0736,NL,GM0736,De Ronde Venen,44457.0,380.167414,116.940585,2020,NL_GM0736,"POLYGON ((3971807.853 3240253.336, 3969495.564..."
...,...,...,...,...,...,...,...,...,...,...
84387,NL_GM0340,NL,GM0340,Rhenen,20125.0,459.723029,43.776358,2020,NL_GM0340,"POLYGON ((4019387.627 3218445.157, 4019813.662..."
84389,NL_GM0342,NL,GM0342,Soest,46607.0,1002.870148,46.473614,2020,NL_GM0342,"POLYGON ((3995813.131 3239542.656, 3997765.403..."
84390,NL_GM0344,NL,GM0344,Utrecht,357694.0,3602.113048,99.301159,2020,NL_GM0344,"POLYGON ((3982408.099 3224453.497, 3981539.501..."
84391,NL_GM0345,NL,GM0345,Veenendaal,66491.0,3357.672736,19.802704,2020,NL_GM0345,"POLYGON ((4014443.278 3223871.992, 4015180.253..."


In [70]:
# Match LAU and NUTS using this work: https://edjnet.github.io/lau_centres/lau_nuts.html

LAU_NUTS_match = pd.read_csv(r'p:\11207608-coclico\FASTTRACK_DATA\XX_NUTS\lau_2020_nuts_2021_concordance_by_geo.csv')
# Drop unneeded columns
LAU_NUTS_match = LAU_NUTS_match.drop(columns=['gisco_id','country','lau_id','lau_name','population','area_km2','year'])
LAU_NUTS_match = LAU_NUTS_match.rename(columns={'fid': 'FID'})

In [57]:
LAU_data

,GISCO_ID,CNTR_CODE,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
0,CZ_579475,CZ,579475,Libňatov,363.0,62.240158,5.832247,2020,CZ_579475,"POLYGON ((4747705.585 3057545.260, 4746836.524..."
1,CZ_579483,CZ,579483,Libotov,178.0,41.774759,4.260946,2020,CZ_579483,"POLYGON ((4731662.418 3049054.578, 4732297.093..."
2,CZ_579491,CZ,579491,Vranov,181.0,37.408873,4.838424,2020,CZ_579491,"POLYGON ((4540137.623 2963864.496, 4538706.113..."
3,CZ_579505,CZ,579505,Malá Úpa,142.0,5.318510,26.699207,2020,CZ_579505,"POLYGON ((4730399.261 3087673.334, 4730914.253..."
4,AT_41004,AT,41004,Eggendorf im Traunkreis,1041.0,116.481747,8.937023,2020,AT_41004,"POLYGON ((4631543.792 2788412.230, 4630476.583..."
...,...,...,...,...,...,...,...,...,...,...
98608,SK_599816,SK,599816,Košice - mestská časť Nad jazerom,24443.0,6686.211551,3.655732,2020,SK_599816,"POLYGON ((5149120.507 2905279.350, 5150154.014..."
98609,SK_599824,SK,599824,Košice - mestská časť Juh,22692.0,2324.881955,9.760496,2020,SK_599824,"POLYGON ((5144451.048 2904986.166, 5144591.047..."
98610,SK_599841,SK,599841,Košice - mestská časť Šaca,5969.0,124.672777,47.877333,2020,SK_599841,"POLYGON ((5141041.890 2900350.394, 5144604.564..."
98611,SK_599859,SK,599859,Košice - mestská časť Poľov,1219.0,94.049511,12.961258,2020,SK_599859,"POLYGON ((5141041.890 2900350.394, 5141299.765..."


In [119]:
# Merge the two dataframes
LAU_NUTS_data = LAU_data.merge(LAU_NUTS_match,on='FID')

# Reorder columns
# Retrieve column names
cols = LAU_NUTS_data.columns

# Move columns
cols = cols.insert([2],cols[-2:])

# Drop the old ones
cols = cols[:-2]

# Store in final form
LAU_NUTS_data = LAU_NUTS_data[cols]

# Write geodataframe to parquet
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FASTTRACK_DATA")
ds_dir = coclico_data_dir.joinpath('XX_LAU')
out_file= ds_dir.joinpath('LAU_2020_NUTS_2021_01M_3035.parquet')

# altered_lau_data.to_parquet(out_file_altered)
LAU_NUTS_data.to_parquet(out_file)


In [120]:
LAU_NUTS_data

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry
0,CZ_579475,CZ,CZ05,CZ052,579475,Libňatov,363.0,62.240158,5.832247,2020,CZ_579475,"POLYGON ((4747705.585 3057545.260, 4746836.524..."
1,CZ_579483,CZ,CZ05,CZ052,579483,Libotov,178.0,41.774759,4.260946,2020,CZ_579483,"POLYGON ((4731662.418 3049054.578, 4732297.093..."
2,CZ_579491,CZ,CZ03,CZ032,579491,Vranov,181.0,37.408873,4.838424,2020,CZ_579491,"POLYGON ((4540137.623 2963864.496, 4538706.113..."
3,CZ_579505,CZ,CZ05,CZ052,579505,Malá Úpa,142.0,5.318510,26.699207,2020,CZ_579505,"POLYGON ((4730399.261 3087673.334, 4730914.253..."
4,AT_41004,AT,AT31,AT312,41004,Eggendorf im Traunkreis,1041.0,116.481747,8.937023,2020,AT_41004,"POLYGON ((4631543.792 2788412.230, 4630476.583..."
...,...,...,...,...,...,...,...,...,...,...,...,...
98608,SK_599816,SK,SK04,SK042,599816,Košice - mestská časť Nad jazerom,24443.0,6686.211551,3.655732,2020,SK_599816,"POLYGON ((5149120.507 2905279.350, 5150154.014..."
98609,SK_599824,SK,SK04,SK042,599824,Košice - mestská časť Juh,22692.0,2324.881955,9.760496,2020,SK_599824,"POLYGON ((5144451.048 2904986.166, 5144591.047..."
98610,SK_599841,SK,SK04,SK042,599841,Košice - mestská časť Šaca,5969.0,124.672777,47.877333,2020,SK_599841,"POLYGON ((5141041.890 2900350.394, 5144604.564..."
98611,SK_599859,SK,SK04,SK042,599859,Košice - mestská časť Poľov,1219.0,94.049511,12.961258,2020,SK_599859,"POLYGON ((5141041.890 2900350.394, 5141299.765..."
